# Kerchunk the AORC-OWP NetCDF files

In [ ]:
import fsspec
import xarray as xr
import ujson   # fast json
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr
from pathlib import Path

In [ ]:
fs_read = fsspec.filesystem('file')

In [ ]:
fs_write = fsspec.filesystem('file')

In [ ]:
nc_files = fs_read.glob('/caldera/hytest_scratch/scratch/rsignell/nc_files/200002/*.nc4') 

len(nc_files)

In [ ]:
from dask.distributed import LocalCluster, Client

cluster = LocalCluster(threads_per_worker=1)

client = Client(cluster)

client

In [ ]:
json_dir = '/caldera/hytest_scratch/scratch/rsignell/json_files'

In [ ]:
nc_files[0]

In [ ]:
#cluster.close()

In [ ]:
def gen_json(u):
    with fs_read.open(u, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, u, inline_threshold=300)
        fstem = Path(u).stem 
        outf = f'{json_dir}/{fstem}.json'
        print(outf)
        with fs_write.open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

In [ ]:
so = dict(mode='rb', default_fill_cache=False, default_cache_type='first')

#### Test one file

In [ ]:
%%time 
gen_json(nc_files[0])

In [ ]:
single_json = '/caldera/hytest_scratch/scratch/rsignell/json_files/AORC-OWP_2000020100z.json'

In [ ]:
%%time
fs = fsspec.filesystem("reference", fo=single_json, ref_storage_args={'skip_instance_cache':True},
                       )

In [ ]:
fs.ls('PRES_surface')

In [ ]:
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs={'consolidated':False}, chunks={})

In [ ]:
ds

#### Use Dask bag to compute the JSONs in parallel

In [ ]:
import dask.bag as db

In [ ]:
cluster.workers

In [ ]:
b = db.from_sequence(nc_files, npartitions=80)

In [ ]:
b1 = b.map(gen_json)

In [ ]:
%%time
_ = b1.compute(retries=10)

In [ ]:
json_list = fs_write.glob(f'{json_dir}/*.json')  

In [ ]:
print(len(json_list))